In [1]:
!pip3 install numpy==1.15.4

     |████████████████████████████████| 13.9MB 233kB/s 
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have numpy 1.15.4 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.15.4 which is incompatible.
ERROR: plotnine 0.6.0 has requirement numpy>=1.16.0, but you'll have numpy 1.15.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: astropy 4.0.1.post1 has requirement numpy>=1.16, but you'll have numpy 1.15.4 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5


In [1]:
import pandas as pd
import numpy as np
import re
import json
import pickle

import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
from torch.optim.lr_scheduler import LambdaLR

from tqdm import tqdm
from tqdm import tqdm_notebook
from tqdm import notebook

from sklearn.model_selection import train_test_split

import h5py
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


# Load Data

In [3]:
cd drive/My\ Drive

/content/drive/My Drive


In [4]:
ls

 CNN_character.ipynb         embedding_matrix.h5       model
 CNN_word_embedding.ipynb    embedding_matrix.tar.gz   name_category.h5
'Colab Notebooks'/           embeddings_keywords.h5    word_nparray.h5
 data_conv_experimental.h5   keywords.pkl              word_nparray.tar.gz


In [5]:
def labels_map(y):
    S=set(y)
    num=len(S)
    dic={word:i for i, word in enumerate(S)}
    
    labels=y.apply(lambda val: dic[val])
    
    return labels.values

### all names and categories

In [ ]:
df_all=pd.read_hdf('name_category.h5',key='data',mode='r')

In [ ]:
num_classes=len(set(df_all['category']))

In [ ]:
df_all['category'].value_counts()[:10]

Adviesbureaus - organisatieadvies en management    86212
Bouwbedrijf                                        45854
Diversen                                           29561
Computersoftware                                   24150
Kapper                                             22471
Adviesbureaus                                      19873
Schoonheidsspecialiste                             19279
Adviesbureaus - automatisering                     19058
Administratiekantoor                               16944
Reclame                                            16448
Name: category, dtype: int64

### embedding matrix and sequences

In [6]:
f=h5py.File('embedding_matrix.h5','r')
embedding_matrix=f['embedding_matrix']

g=h5py.File('word_nparray.h5','r')
word_nparray=g['word_nparray']
word_nparray=word_nparray[()]

### small data-set

In [ ]:
lst=['Bouwbedrijf','Computersoftware','Kapper','Reclame','Administratiekantoor','Adviesbureaus']

In [ ]:
df_reduced=df_all[df_all['category'].isin(lst)].copy()
df_reduced.loc[:,'name']=df_reduced['name'].str.lower()
idx=df_all['category'].isin(lst)

In [ ]:
word_reduced=word_nparray[idx.values,:]

In [ ]:
num_classes=len(set(df_reduced['category']))
labels=labels_map(df_reduced['category'])

In [ ]:
num_classes=len(set(df_all['category']))
labels=labels_map(df_all['category'])

In [ ]:
num_classes

6

## Names and more general categories

In [7]:
df=pd.read_hdf('data_conv_experimental.h5',key='data',mode='r')

In [ ]:
df['category0'].value_counts()

opleiding          355211
diensten           262805
winkel             259861
vrije tijd         140906
gezondheidszorg    136865
gereedschap        127734
regering            99355
reizen              25868
boodschappen         3901
Name: category0, dtype: int64

In [8]:
lst=['gereedschap','regering']
idx=  df['category0'].isin(lst)
df_reduced=df[idx].copy()
df_reduced.loc[:,'name']=df_reduced['name'].str.lower()

word_reduced=word_nparray[idx.values,:]
num_classes=len(set(df_reduced['category0']))
labels=labels_map(df_reduced['category0'])

# Models: Conv1D, LSTM, FNN

### Conv1D Word embedding

In [9]:
class Conv1D(nn.Module):
    def __init__(self,embedding_data,max_length,output_dim):
        super(Conv1D,self).__init__()
        
        self.embedding_dim=embedding_data.shape[1]
        self.embed=nn.Embedding.from_pretrained(torch.from_numpy(embedding_data).float())
        self.conv1=nn.Sequential(nn.Conv1d(self.embedding_dim,200,kernel_size=3,padding=0),nn.ReLU(),nn.MaxPool1d(2))
        self.conv2=nn.Sequential(nn.Conv1d(200,200,kernel_size=1,padding=0),nn.ReLU(),nn.MaxPool1d(2))
        self.conv3=nn.Sequential(nn.Conv1d(200,200,kernel_size=1,padding=0),nn.ReLU())
        self.max_length=max_length

        input_shape=(128,self.max_length,self.embedding_dim)
        self.output_conv_dim = self.get_conv_output(input_shape)
        
        self.drop_input=nn.Dropout2d(0.2)
        self.fc1 = nn.Sequential(nn.Linear(self.output_conv_dim, 500), nn.ReLU(),nn.Dropout(0.3)  )
        self.fc2 = nn.Sequential( nn.Linear(500, 500), nn.ReLU(),nn.Dropout(0.3) )
        self.fc3=nn.Linear(500,output_dim)
        
    def get_conv_output(self,shape):
        x = torch.rand(shape)
        x = x.transpose(1, 2)
        x = self.conv1(x)
        x = self.conv2(x)
        #x = self.conv3(x)
        x = x.view(x.size(0), -1)
        out_dim = x.size(1)
        return out_dim
        
        
        
    def forward(self,x):
        x=x[:,:self.max_length]
        x=self.embed(x)
        x = x.transpose(1, 2)
        x=self.drop_input(x)
        x = self.conv1(x)
        x = self.conv2(x)
        #x = self.conv3(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [77]:
class Conv1D_concat(nn.Module):
    def __init__(self,embedding_data,max_length,output_dim):
        super(Conv1D_concat,self).__init__()
        
        self.embedding_dim=embedding_data.shape[1]
        self.embed=nn.Embedding.from_pretrained(torch.from_numpy(embedding_data).float())
        self.conv1=nn.Sequential(nn.Conv1d(self.embedding_dim,300,kernel_size=1,padding=0),nn.ReLU(),nn.MaxPool1d(2))
        self.conv2=nn.Sequential(nn.Conv1d(300,300,kernel_size=2,padding=0),nn.ReLU(),nn.MaxPool1d(2))
        self.conv3=nn.Sequential(nn.Conv1d(300,300,kernel_size=3,padding=0),nn.ReLU(),nn.MaxPool1d(2))
        self.max_length=max_length

        input_shape=(128,self.max_length,self.embedding_dim)
        self.output_conv_dim = self.get_conv_output(input_shape)
        
        self.drop_input=nn.Dropout2d(0.4)
        self.fc1 = nn.Sequential(nn.Linear(self.output_conv_dim, 1000), nn.ReLU(),nn.Dropout(0.3)  )
        self.fc2 = nn.Sequential( nn.Linear(1000, 1000), nn.ReLU(),nn.Dropout(0.3) )
        self.fc3=nn.Linear(1000,output_dim)
        
    def get_conv_output(self,shape):
        x = torch.rand(shape)
        x = x.transpose(1, 2)
        x = self.conv1(x)
        y = self.conv2(x)
        #z = self.conv3(x)
        x = x.view(x.size(0), -1)
        y = y.view(y.size(0), -1)
        #z = z.view(z.size(0), -1)
        x=torch.cat((x,y),1)
        out_dim = x.size(1)
        return out_dim
        
    def forward(self,x):
        x=x[:,:self.max_length]
        x=self.embed(x)
        x = x.transpose(1, 2)
        x=self.drop_input(x)
        x = self.conv1(x)
        y = self.conv2(x)
        #z = self.conv3(x)
        x = x.view(x.size(0), -1)
        y = y.view(y.size(0), -1)
        #z = z.view(z.size(0), -1)
        x=torch.cat((x,y),1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

### Conv1D character

In [10]:
max_length=40
alphabet="abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
args={'alphabet':alphabet,'number_of_characters':len(alphabet),'max_length':max_length,'dropout_input':0.2,'number_of_classes':num_classes}

In [56]:
class CharacterLevelCNN(nn.Module):
    def __init__(self, args):
        super(CharacterLevelCNN, self).__init__()

        # define conv layers
        self.drop_input=nn.Dropout2d(0.3)
        self.conv1 = nn.Sequential(nn.Conv1d(args['number_of_characters'],
                                             300,
                                             kernel_size=6,
                                             padding=0),
                                   nn.ReLU(),
                                   nn.MaxPool1d(2)
                                   )

        self.conv2 = nn.Sequential(nn.Conv1d(300, 300, kernel_size=10, padding=0),
                                   nn.ReLU(),
                                   nn.MaxPool1d(2)
                                   )
      
        self.conv3 = nn.Sequential(nn.Conv1d(300, 300, kernel_size=7, padding=0),
                                   nn.ReLU()
                                   )
        
        
        # compute the  output shape after forwarding an input to the conv layers

        input_shape = (128,
                       args['max_length'],
                       args['number_of_characters'])
        
        self.output_dimension = self._get_conv_output(input_shape)

        # define linear layers

        self.fc1 = nn.Sequential(
            nn.Linear(self.output_dimension, 500),
            nn.ReLU(),nn.Dropout(0.2)
            
        )

        self.fc2 = nn.Sequential(
            nn.Linear(500, 500),
            nn.ReLU()
            
        )

        self.fc3 = nn.Linear(500, args['number_of_classes'])

        # initialize weights

        self._create_weights()

    # utility private functions

    def _create_weights(self, mean=0.0, std=0.05):
        for module in self.modules():
            if isinstance(module, nn.Conv1d) or isinstance(module, nn.Linear):
                module.weight.data.normal_(mean, std)


    def _get_conv_output(self, shape):
        x = torch.rand(shape)
        x = x.transpose(1, 2)
        x = self.conv1(x)
        x = self.conv2(x)
        #x = self.conv3(x)
        x = x.view(x.size(0), -1)
        output_dimension = x.size(1)
        return output_dimension

    # forward

    def forward(self, x):
        x=self.drop_input(x)
        x = x.transpose(1, 2)
        x = self.conv1(x)
        x = self.conv2(x)
        #x = self.conv3(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [12]:
class MyDataset(Dataset):
    def __init__(self, texts, labels, args):
        self.texts = texts
        self.labels = labels
        self.length = len(self.texts)

        self.vocabulary = args['alphabet'] 
        self.number_of_characters = args['number_of_characters'] 
        self.max_length = args['max_length']
        self.identity_mat = np.identity(self.number_of_characters)

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        raw_text = self.texts[index]

        data = np.array([self.identity_mat[self.vocabulary.index(i)] for i in list(raw_text) if i in self.vocabulary],
                        dtype=np.float32)
        if len(data) > self.max_length:
            data = data[:self.max_length]
        elif 0 < len(data) < self.max_length:
            data = np.concatenate(
                (data, np.zeros((self.max_length - len(data), self.number_of_characters), dtype=np.float32)))
        elif len(data) == 0:
            data = np.zeros(
                (self.max_length, self.number_of_characters), dtype=np.float32)

        label = self.labels[index]
        data = torch.Tensor(data)

        return data, label

### Hybrid model

In [ ]:
class WordCharModel(nn.Module):
  def __init__(self,embed_dim,max_word_length,args,output_dim):
    super(WordCharModel,self).__init__()
    
    self.embedding_dim=embed_dim
    self.max_word_length=max_word_length
    self.out_dim=output_dim
    self.conv1w=nn.Sequential(nn.Conv1d(self.embedding_dim,300,kernel_size=3,padding=0),nn.ReLU(),nn.MaxPool1d(2))
    self.conv2w=nn.Sequential(nn.Conv1d(300,300,kernel_size=1,padding=0),nn.ReLU(),nn.MaxPool1d(2))

    self.drop_input=nn.Dropout2d(0.3)
    self.conv1char = nn.Sequential(nn.Conv1d(args['number_of_characters'],
                                             300,
                                             kernel_size=15,
                                             padding=0),
                                   nn.ReLU(),
                                   nn.MaxPool1d(2)
                                   )

    self.conv2char = nn.Sequential(nn.Conv1d(300, 300, kernel_size=5, padding=0),
                                   nn.ReLU(),
                                   nn.MaxPool1d(2)
                                   )
    
    input_shape=(128,self.max_word_length,self.embedding_dim)
    self.output_word_conv_dim = self.get_word_conv_output(input_shape)
    input_shape = (128,args['max_length'],args['number_of_characters'])
    self.output_char_conv_dim = self.get_char_conv_output(input_shape)

    self.fc1_w = nn.Sequential(
            nn.Linear(self.output_word_conv_dim, 500),
            nn.ReLU(),nn.Dropout(0.2)
            
        )

    self.fc2_w = nn.Sequential(
            nn.Linear(500, 500),
            nn.ReLU()
            
        )

    self.fc3_w = nn.Linear(500, self.out_dim)

    self.fc1_char = nn.Sequential(
            nn.Linear(self.output_char_conv_dim, 500),
            nn.ReLU(),nn.Dropout(0.2)
            
        )

    self.fc2_char = nn.Sequential(
            nn.Linear(500, 500),
            nn.ReLU()
            
        )

    self.fc3_char = nn.Linear(500, self.out_dim)

  def get_word_conv_output(self, shape):
        x = torch.rand(shape)
        x = x.transpose(1, 2)
        x = self.conv1w(x)
        x = self.conv2w(x)
        x = x.view(x.size(0), -1)
        output_dimension = x.size(1)
        return output_dimension

  def get_char_conv_output(self,shape):
        x = torch.rand(shape)
        x = x.transpose(1, 2)
        x = self.conv1char(x)
        x = self.conv2char(x)
        x = x.view(x.size(0), -1)
        output_dimension = x.size(1)
        return output_dimension

  def forward(self,x,y):
        x=self.drop_input(x)
        x = x.transpose(1, 2)
        x = self.conv1char(x)
        x = self.conv2char(x)
        #x = self.conv3(x)
        x = x.view(x.size(0), -1)
        x = self.fc1_char(x)
        x = self.fc2_char(x)
        x = self.fc3_char(x)

        y = y.transpose(1, 2)
        y=self.drop_input(y)
        y = self.conv1w(y)
        y = self.conv2w(y)
        #x = self.conv3(x)
        y = y.view(y.size(0), -1)
        y = self.fc1_w(y)
        y = self.fc2_w(y)
        y = self.fc3_w(y)

        return x+y


In [ ]:
class HybridDataset(Dataset):
    def __init__(self, texts,sequences, embedding_data, labels,max_word_len, args):
        self.texts = texts
        self.sequences=torch.from_numpy(sequences)
        self.labels = labels
        self.length = len(self.texts)
        self.embedding=nn.Embedding.from_pretrained(torch.from_numpy(embedding_data).float())
        self.max_word_length=max_word_len

        self.vocabulary = args['alphabet'] 
        self.number_of_characters = args['number_of_characters'] 
        self.max_length = args['max_length']
        self.identity_mat = np.identity(self.number_of_characters)

        try:
          assert len(texts)==len(sequences)
        except:
          print('texts and sequences have different sizes!')

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        raw_text = self.texts[index]
        seq=self.sequences[index]
        seq=seq[:self.max_word_length]
        embedded_text=self.embedding(seq)

        data = np.array([self.identity_mat[self.vocabulary.index(i)] for i in list(raw_text) if i in self.vocabulary],
                        dtype=np.float32)
        if len(data) > self.max_length:
            data = data[:self.max_length]
        elif 0 < len(data) < self.max_length:
            data = np.concatenate(
                (data, np.zeros((self.max_length - len(data), self.number_of_characters), dtype=np.float32)))
        elif len(data) == 0:
            data = np.zeros(
                (self.max_length, self.number_of_characters), dtype=np.float32)

        label = self.labels[index]
        data = torch.Tensor(data)

        return embedded_text,data, label

# Training scripts:

In [10]:
def train(model, train_loader, epochs, criterion, optimizer, device):
    L=len(train_loader)
    for epoch in range(1, epochs + 1):
        model.train() # Make sure that the model is in training mode.

        total_loss = 0

        for batch in notebook.tqdm(train_loader):
            # get data
            batch_x, batch_y = batch
            
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            optimizer.zero_grad()

            # get predictions from model
            y_pred = model(batch_x)
        
            # perform backprop
            loss = criterion(y_pred, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.data.item()
        
        print("Epoch: {}, Loss: {}".format(epoch, total_loss / L))

In [14]:
def train_hybrid(model, train_loader, epochs, criterion, optimizer, device):
    L=len(train_loader)
    for epoch in range(1, epochs + 1):
        model.train() # Make sure that the model is in training mode.

        total_loss = 0

        for batch in notebook.tqdm(train_loader):
            # get data
            batch_word,batch_char, batch_y = batch
            
            batch_word = batch_word.to(device)
            batch_char=batch_char.to(device)
            batch_y = batch_y.to(device)

            optimizer.zero_grad()

            # get predictions from model
            y_pred = model(batch_char,batch_word)
        
            # perform backprop
            loss = criterion(y_pred, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.data.item()
        
        print("Epoch: {}, Loss: {}".format(epoch, total_loss / L))

# Train/Test datasets preparation

### Word_embedding datasets

In [11]:
indices=np.arange(word_reduced.shape[0])
np.random.shuffle(indices)
data_x=word_reduced[indices]
data_y=labels[indices]

nb_validation=int(0.2*word_reduced.shape[0])

x_train=data_x[:-nb_validation]
y_train=data_y[:-nb_validation]

x_test=data_x[-nb_validation:]
y_test=data_y[-nb_validation:]

x_train=torch.from_numpy(x_train)
y_train=torch.from_numpy(y_train)

train_ds = torch.utils.data.TensorDataset(x_train, y_train)
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=128)

x_test=torch.from_numpy(x_test)
y_test=torch.from_numpy(y_test)

test_ds = torch.utils.data.TensorDataset(x_test, y_test)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=128)

### character conv1d datasets

In [16]:
data_x=df_reduced.iloc[indices].copy()['name'].str.lower().values

nb_validation=int(0.2*df_reduced.shape[0])

x_train=data_x[:-nb_validation]
y_train=data_y[:-nb_validation]

x_test=data_x[-nb_validation:]
y_test=data_y[-nb_validation:]

train_char_ds=MyDataset(x_train,y_train,args)
test_char_ds=MyDataset(x_test,y_test,args)
test_char_dl=torch.utils.data.DataLoader(test_char_ds, batch_size=128)
train_char_dl=torch.utils.data.DataLoader(train_char_ds, batch_size=128)

### Hybrid dataset preparation

In [ ]:
indices=np.arange(word_reduced.shape[0])
np.random.shuffle(indices)
data_x=word_reduced[indices]
data_char=df_reduced.iloc[indices].copy()['name'].str.lower().values
data_y=labels[indices]

nb_validation=int(0.2*df_reduced.shape[0])
x_train_word=data_x[:-nb_validation]
x_train_char=data_char[:-nb_validation]
y_train=data_y[:-nb_validation]

x_test_word=data_x[-nb_validation:]
x_test_char=data_char[-nb_validation:]
y_test=data_y[-nb_validation:]

train_hyb_ds=HybridDataset(x_train_char,x_train_word,embedding_matrix[()],y_train,6,args)
test_hyb_ds=HybridDataset(x_test_char,x_test_word,embedding_matrix[()],y_test,6,args)

test_hyb_dl=torch.utils.data.DataLoader(test_hyb_ds, batch_size=512)
train_hyb_dl=torch.utils.data.DataLoader(train_hyb_ds, batch_size=512)

## Accuracy and Confusion matrix

In [12]:
 def accuracy(model,val_dl):
  model.eval()
  with torch.no_grad():
        accu=0
        for batch in val_dl:
            val_X,val_y=batch
            val_X=val_X.to(device)
            val_y=val_y.to(device)
            x=model(val_X)
            probs=F.softmax(x,dim=1)
            preds=probs.argmax(dim=1)
            accu+=(preds==val_y).sum().item()
    
  accu=accu/len(val_dl.dataset)*100
  return accu

def accuracy_lstm(model,val_dl):
  model.eval()
  with torch.no_grad():
        accu=0
        for batch in val_dl:
            val_X,val_len, val_y=batch
            val_X=val_X.to(device)
            val_y=val_y.to(device)
            val_len=val_len.to(device)

            x=model(val_X,val_len)
            probs=F.softmax(x,dim=1)
            preds=probs.argmax(dim=1)
            accu+=(preds==val_y).sum().item()
    
  accu=accu/len(val_dl.dataset)*100
  return accu
  
def confusion_matrix(model,val_dl,nb_classes):
  conf_matrix = torch.zeros(nb_classes, nb_classes)
  model.eval()
  with torch.no_grad():
      for i, (inputs, classes) in enumerate(val_dl):
          inputs = inputs.to(device)
          classes = classes.to(device)
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          for t, p in zip(classes.view(-1), preds.view(-1)):
                  conf_matrix[t.long(), p.long()] += 1
  conf_sum=conf_matrix.sum(1)
  conf_sum=conf_sum.reshape(-1,1)
  print(conf_matrix/conf_sum*100)

# Training

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device {}.".format(device))

Using device cuda.


### training word_embedding Conv1D

In [ ]:
model_word=Conv1D(embedding_matrix[()],max_length=6,output_dim=num_classes).to(device)

In [78]:
model_word=Conv1D_concat(embedding_matrix[()],max_length=6,output_dim=num_classes).to(device)

In [79]:
model_word.output_conv_dim

1200

In [80]:
optimizer = optim.Adam(model_word.parameters(),lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

In [81]:
train(model_word, train_dl, 10, loss_fn,optimizer, device)


Epoch: 1, Loss: 0.5778375762659059



Epoch: 2, Loss: 0.5434043391909398



Epoch: 3, Loss: 0.5325224297986904



Epoch: 4, Loss: 0.5266920044598445



Epoch: 5, Loss: 0.5219771919326044



Epoch: 6, Loss: 0.5178157492632597



Epoch: 7, Loss: 0.5148894683156215



Epoch: 8, Loss: 0.5122223207648371



Epoch: 9, Loss: 0.5095893888616226



Epoch: 10, Loss: 0.5058301994078596


In [21]:
len(train_ds)

181672

In [84]:
accuracy(model_word,train_dl)

75.34182482716105

In [83]:
accuracy(model_word,test_dl)

72.86478631349495

### training character Conv1D

In [57]:
model_char=CharacterLevelCNN(args).to(device)

In [58]:
model_char.output_dimension

1200

In [69]:
optimizer = optim.Adam(model_char.parameters(),lr=0.0001)
loss_fn = torch.nn.CrossEntropyLoss()

In [70]:
train(model_char, train_char_dl, 10, loss_fn,optimizer, device)


Epoch: 1, Loss: 0.5279927897411333



Epoch: 2, Loss: 0.5277202590460509



Epoch: 3, Loss: 0.5271819317424801



Epoch: 4, Loss: 0.5253559694113866



Epoch: 5, Loss: 0.5261793022424402



Epoch: 6, Loss: 0.5254154719517264



Epoch: 7, Loss: 0.5265084109885592



Epoch: 8, Loss: 0.5259950847273142



Epoch: 9, Loss: 0.5251204524451578



Epoch: 10, Loss: 0.5256329130538753


In [71]:
accuracy(model_char,train_char_dl)

74.11378748513805

In [72]:
accuracy(model_char,test_char_dl)

72.25928617037674

### Hybrid Training

In [ ]:
embed_dim=embedding_matrix.shape[1]

In [ ]:
model_hyb=WordCharModel(embed_dim,6,args,num_classes).to(device)

In [ ]:
optimizer = optim.Adam(model_hyb.parameters(),lr=0.00001)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
train_hybrid(model_hyb, train_hyb_dl, 10, loss_fn,optimizer, device)


Epoch: 1, Loss: 0.34845314877170125



Epoch: 2, Loss: 0.34922606406504647



Epoch: 3, Loss: 0.34783330474173624



Epoch: 4, Loss: 0.3479758620357209



Epoch: 5, Loss: 0.34810311279513617



Epoch: 6, Loss: 0.3471368134877328



Epoch: 7, Loss: 0.3467352404786449



Epoch: 8, Loss: 0.3478340166464947



Epoch: 9, Loss: 0.34736690666686976



Epoch: 10, Loss: 0.3467338071152734


### Hybrid model accuracy

In [ ]:
def accuracy_hybrid_model(model,val_dl):
  model.eval()
  with torch.no_grad():
        accu=0
        for batch in val_dl:
            val_w,val_char, val_y=batch
            val_w=val_w.to(device)
            val_char=val_char.to(device)
            val_y=val_y.to(device)

            x=model(val_char,val_w)
            probs=F.softmax(x,dim=1)
            preds=probs.argmax(dim=1)
            accu+=(preds==val_y).sum().item()
    
  accu=accu/len(val_dl.dataset)*100
  return accu

In [ ]:
accuracy_hybrid_model(model_hyb,train_hyb_dl)

85.6494101941899

In [ ]:
accuracy_hybrid_model(model_hyb,test_hyb_dl)

NameError: ignored

# Hybrid prediction

In [ ]:
def accuracy_hybrid(model_w,model_c,val_w_dl,val_c_dl,weight):
  model_w.eval()
  model_c.eval()
  with torch.no_grad():
        accu=0
        for batch_w,batch_c in zip(val_w_dl,val_c_dl):
            val_w_X,val_w_y=batch_w
            val_w_X=val_w_X.to(device)
            val_w_y=val_w_y.to(device)

            val_c_X,val_c_y=batch_c
            val_c_X=val_c_X.to(device)
            val_c_y=val_c_y.to(device)

            x=model_w(val_w_X)
            y=model_c(val_c_X)

            probs_w=F.softmax(x,dim=1)
            probs_c=F.softmax(y,dim=1)
            probs=weight*probs_w+(1-weight)*probs_c

            preds=probs.argmax(dim=1)
            accu+=(preds==val_w_y).sum().item()
    
  accu=accu/len(val_w_dl.dataset)*100
  return accu

def accuracy_hybrid_2(model_w,model_c,val_w_dl,val_c_dl,ws):
  model_w.eval()
  model_c.eval()
  with torch.no_grad():
        accu=0
        for batch_w,batch_c in zip(val_w_dl,val_c_dl):
            val_w_X,val_w_y=batch_w
            val_w_X=val_w_X.to(device)
            val_w_y=val_w_y.to(device)

            val_c_X,val_c_y=batch_c
            val_c_X=val_c_X.to(device)
            val_c_y=val_c_y.to(device)

            x=model_w(val_w_X)
            y=model_c(val_c_X)
            z=ws[0]*x+ws[1]*y

            probs=F.softmax(z,dim=1)
            preds=probs.argmax(dim=1)
            accu+=(preds==val_w_y).sum().item()
    
  accu=accu/len(val_w_dl.dataset)*100
  return accu

In [ ]:
accuracy_hybrid(model_word,model_char,test_dl,test_char_dl,0.7)

76.55109602884484

In [ ]:
accuracy_hybrid_2(model_word,model_char,test_dl,test_char_dl,[0.7,0.2])

76.43007747682557

In [ ]:
70.09057225195554

In [ ]:
accus=[]
for w in notebook.tqdm(range(30,80)):
  ac=accuracy_hybrid_2(model_word,model_char,test_dl,test_char_dl,w/100)
  accus.append(ac)